In [1]:
import sys
sys.path.insert(0,'../FacebookCrawler/')
sys.path.insert(0,'../database/')
import time
from facebook_graph_api import *
import utils.constants
import pandas as pd
from cassandra.util import uuid_from_time
from threading import Thread
import models
import db_cassandra
from cassandra.query import dict_factory
from cassandra.cqlengine import columns, connection
import cassandra.util
from datetime import datetime

FORMAT = '%(asctime)-15s %(clientip)s %(user)-8s %(message)s'
logging.basicConfig(level=logging.ERROR)
LOGGER.info("business discovery...")


In [2]:
def get_influencer_profiles(begin, end, cass_session, fb_graph):
    connection.set_session(cass_session)
    list_business_discovery_all = []
    # read instagram id 
#     path = join(dirname(dirname(os.path.abspath(__file__))),constants.instagram_id_path_file)
    path = '../%s'%constants.instagram_id_path_file
    with open(path, "r") as read_file:
        list_id = read_file.read().split('\n')
#     f = open(writer_file, 'a')
    i = 0
    for instagram_id in list_id[begin:end]:
        print('len list %s'%len(list_id[begin:end]))
        time.sleep(2)
        print('sleeping %s'%constants.TIME_SLEEP)
        i = i + 1
        print("====== %s ======" %(i))
        try:
            business_discovery_all = fb_graph.get_business_discovery_all(media_count = constants.MAX_MEDIA, username = instagram_id)#17841401610104138, 17841407517779759
            if 'error' not in business_discovery_all:
                print(business_discovery_all['username'])
                # create list to convert into df and save file
                # list_business_discovery_all.append(self.parser_influencer_data(business_discovery_all))
                dict_result = fb_graph.parser_influencer_data(business_discovery_all)
                dict_result['ts'] = uuid_from_time(time.time())
                event = models.UserBusinessDiscovery(**dict_result)
                event.save()
            else:
                print('error: %s'%instagram_id)
        except Exception as ex:
            print(ex)
            pass

In [3]:
# constants.KEYSPACE_NAME = 'aify_recommend'
constants.KEYSPACE_NAME = 'test'

In [155]:
import facebook_graph_api
import argparse, time



def thread1(threadname,fb_graph, begin, end,file_name):
    print('start %s....'%threadname)
    fb_graph.get_influencer_profiles(begin, end,file_name)
    print('finished %s....'%threadname)


if __name__=="__main__":
    fb_graph = facebook_graph_api.FacebookGraphApi('../FacebookCrawler/instagram_token.csv')
    (fb_graph.access_token,fb_graph.user_id) = constants.TOKENS[-1] 

    thread1 = Thread( target=thread1, args=("Thread-1",fb_graph, 2500, 3500,'3500.txt') )
#     thread2 = Thread( target=thread2, args=("Thread-2",fb_graph, 3000, 4000,'4000.txt' ) )

    cass_db = db_cassandra.db_cassandra(['172.33.47.9'],dict_factory, port_ = 9042)
#     thread1.start()

/home/anh_lbt/atvn/sw_follou/recommender_api/test


NoHostAvailable: ("Unable to connect to any servers using keyspace 'test'", ['172.33.47.9'])

In [40]:
# get_influencer_profiles(10, 20, cass_db.session, fb_graph)

In [41]:
# test insert into cassandra

# ['id', 'biography', 'followers_count', 'media_count', 'name', 'username', 'profile_picture_url', 'comments_count', 'likes_count']
business_discovery_all = fb_graph.get_business_discovery_all(media_count = constants.MAX_MEDIA, username = 'ludovica_ferlito')#17841401610104138, 17841407517779759
if 'error' not in business_discovery_all:
#     print(business_discovery_all['username'])
    res = fb_graph.parser_influencer_data(business_discovery_all)
#     print(res)


In [156]:
from cassandra.cqlengine.models import Model
from cassandra.cqlengine import columns, connection
from cassandra.cluster import Cluster
from cassandra.query import dict_factory
from cassandra.cqlengine.columns import *
from cassandra.cqlengine.usertype import UserType



import pandas as pd
import os

def read_multi_file_to_df(dir_files = './data_api_insta/'):
    df_inf = pd.DataFrame(index=None)
    for file in os.listdir(dir_files):
        if file.endswith('.txt'):
            data = pd.read_csv(os.path.join(dir_files, file),sep='\t', header=0)
#             df = pd.DataFrame.from_dict(data, orient='columns')
            df_inf= pd.concat([data, df_inf], axis=0)
    return df_inf


class UserBusinessDiscovery(Model):
    __keyspace__ = constants.KEYSPACE_NAME
    __table_name__ = "users_business_discovery"
    id = columns.BigInt(primary_key=True)
    name = columns.Text()
    biography = columns.Text()
    media_count = columns.Integer()
    comments_count = columns.Integer()
    likes_count = columns.Integer()
    followers_count = columns.Integer()
    username = columns.Text()
    profile_picture_url = columns.Text()
    updated_date = DateTime()
#     ts = columns.UUID(primary_key=True)
#     categories = columns.List(columns.Text())
#     account_type = columns.Integer()

    

In [43]:

# #get session
_session = Cluster(contact_points =['172.33.47.9'], port=9042).connect('aify_recommend')
_session.row_factory = dict_factory
connection.set_session(_session)
constants.KEYSPACE_NAME = 'aify_recommend'

# #preprocessing
# df_api = read_avarta_urls_to_df()
# df_api['biography']= df_api['biography'].fillna("")
# df_api['name']= df_api['name'].fillna("")
# df_api['website'] = df_api['website'].fillna("")
# #convert to dict and insert into db
# for i in range(df_api.shape[0]):
#     res = df_api.iloc[i].to_dict()
#     res['ts'] = uuid_from_time(time.time())
#     event = UserBusinessDiscovery(**res)
#     event.save()

In [10]:
# class Person(Model):
#     first_name  = columns.Text(primary_key=True)
#     last_name = columns.Text()

# kevin = Person.create(first_name="Kevin", last_name="Deldycke")
# dict(kevin)  # returns {'first_name': 'Kevin', 'last_name': 'Deldycke'}
# kevin['first_name']  # returns 'Kevin'
# kevin.keys()  # returns ['first_name', 'last_name']
# kevin.values()  # returns ['Kevin', 'Deldycke']
# kevin.items()  # returns [('first_name', 'Kevin'), ('last_name', 'Deldycke')]

# kevin['first_name'] = 'KEVIN5000'  # changes the models first name


# CREATE TABLE IF NOT EXISTS aify_recommend.Person (
#     id timeuuid,
#     user_name text,
#     first_name text,
#     last_name text,
#     updated_date timestamp,
#     PRIMARY KEY(id, user_name)
# ) WITH CLUSTERING ORDER BY (id DESC);



import time
from cassandra.cqlengine.columns import UUID
from cassandra.util import uuid_from_time
#using the person model from earlier:
class Person(Model):
    id = columns.UUID(primary_key=True)
    user_name = columns.Text(primary_key=True)
    first_name  = columns.Text()
    last_name = columns.Text()
    updated_date = columns.DateTime()
    

# person = Person(id = uuid_from_time(time.time()),user_name = 'anhlbt', first_name='Blake', last_name='Eggleston')

In [11]:
person = Person(id = uuid_from_time(time.time()),user_name = 'anhle', first_name='Anh', last_name='Le')
person.save()

Person(id=UUID('f84afc38-afe3-11e8-b2d9-2f4f5f9f82eb'), user_name='anhle', first_name='Anh', last_name='Le', updated_date=None)

In [12]:
# person.objects(id='81833564-a5bd-11e8-a36c-67d94cf8123f', user_name = 'anhlbt').update(last_name='CCBBBBBBBBBBBCCC')
person.objects(id='81833564-a5bd-11e8-a36c-67d94cf8123f', user_name = 'anhlbt').update(last_name='updatedddddddd')

In [18]:
# _session.execute("CREATE INDEX ON aify_recommend.Person(first_name);")

In [4]:
#convert OrderedMapSerializedKey to dict (from cassandra)
from cassandra.util import OrderedMapSerializedKey

def OrderedMapSerializedKeyToDict(map_data): 
    lst = []
    for idx_value, i_value in enumerate(map_data):
        if type(i_value) is OrderedMapSerializedKey:
#             print(i_value) #{'follower': '15.6k', 'link': 'https://instagram.com/martilenia', 'type': 'Instagram Link Click'}
            lst.append(dict(i_value))
    return lst
# OrderedMapSerializedKeyToDict(row[0])


def map_to_dict_factory_2(dict_):
    try:
        for key, value in dict_.items():
            if isinstance(value, OrderedMapSerializedKey):
                d = dict()
                for key2, value2 in dict(value).items():
                    if isinstance(value2, OrderedMapSerializedKey):
                        d[key2] = dict(value2)
                dict_[key] = d    
    except Exception as ex:
        print(ex)
        pass
    return dict_

#return list of dict
def map_to_dict_factory_1(list_of_dict):
    try:
        ls_dic_re = []
        for idx_value, i_value in enumerate(list_of_dict):
            for key, value in i_value.items():
                if isinstance(value, OrderedMapSerializedKey):
                    i_value[key] = dict(value)
            ls_dic_re.append(i_value)
    except Exception as ex:
        print(ex)
        pass
    return ls_dic_re

class type_business_info(UserType):
    id = columns.Text()
    website = columns.Text()
    media_count = columns.Integer()
    followers_count = columns.Integer()
    name = columns.Text()
    username = columns.Text()
    profile_picture_url = columns.Text()
    new_post = columns.Integer(default = 0)

    
class watch_list(Model):
    __keyspace__ = constants.KEYSPACE_NAME
    __table_name__ = "watch_list"
    user_id= columns.Text(primary_key=True)
    watch_user_list= Map(Text(), UserDefinedType(type_business_info))
    viewed_time =columns.DateTime()
    updated_time =columns.DateTime()
    
class recommend_influencer(Model):
    __keyspace__ = constants.KEYSPACE_NAME
    __table_name__ = "recommend_influencer_test"
    influencer_name = Text()
    bio = Text()
    categories = List(Text())
    commentPerPost =Integer()
    demographics = Text()
    engagementRate = Text()
    follower = Integer()
    account_type =Integer()
    likePerPost = Integer()
    location = Text()
    path =Text()
    reach =Text()
    website =Text()
    posts_starting =Integer()
    currency =Text()
    socialAccounts = List(Map(Text(), Text()))
    created =DateTime(default=  datetime.utcnow())
    updated =DateTime(default=  datetime.utcnow())
    activate =DateTime(default=  datetime.utcnow())
    influencer_id = Integer(primary_key=True)
    img_url = Text()
    ts = UUID(primary_key=True)
    

NameError: name 'UserType' is not defined

# update table users_business_discovery

In [ ]:
#

In [157]:
# #get session
_session = Cluster(contact_points =['172.33.47.10'], port=61003).connect('test')
_session.row_factory = dict_factory
connection.set_session(_session)

In [196]:
# return format dataframe when query results from cassandra
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)


# db = db_cassandra.db_cassandra(['172.33.47.9'], pandas_factory, port_ = 9042)

constants.KEYSPACE_NAME = 'aify_db'
db = db_cassandra.db_cassandra(['172.33.47.10'], pandas_factory, port_ = 61003)
db.cluster.connect('aify_db')
    
rows = db.session.execute("select * from %s.recommend_influencer;"%(constants.KEYSPACE_NAME))
df = rows._current_rows

In [43]:
# df.head()

In [44]:
#
df_c = df[['influencer_id', 'account_type', 'categories']]

In [45]:
df_c.head()

,influencer_id,account_type,categories
0,4317,0,"[Fashion, Hairstyles, Makeup, Skincare, Beauty..."
1,3372,0,"[Beauty, Fashion, Fashion Brands, Makeup, Desi..."
2,1584,1,"[Fashion, Jewelry, Weddings, Free Spirit]"
3,7034,1,"[Cats, Coffee, Cosmetics]"
4,4830,0,"[Digital Marketing, Digital Media, Fashion, Ph..."


In [414]:
rows = db.session.execute("select * from %s.users_business_discovery;"%(constants.KEYSPACE_NAME))
df = rows._current_rows

In [415]:
df.head(1)

,id,username,biography,comments_count,followers_count,likes_count,media_count,name,profile_picture_url,updated_date
0,17841403192351597,__twinkl,𖥸 Self Taught #MUE | 27 \n𖥸 #beautyinfluencer...,6331,5712,59383,675,Kristen💄MUE,https://scontent.xx.fbcdn.net/v/t51.2885-15/39...,2018-09-04 22:19:55


In [416]:
def calculate_engagement(followers_count, likes_count, comments_count):
    try:
        if followers_count > 0:
            return (followers_count + likes_count)/comments_count * 1.0
    except Exception as ex:
        print(ex)     

In [419]:
df['engagement'] = list(map(calculate_engagement, df.followers_count, df.likes_count, df.comments_count))

division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero
division by zero


In [420]:
df

,id,username,biography,comments_count,followers_count,likes_count,media_count,name,profile_picture_url,updated_date,engagement
0,17841403192351597,__twinkl,𖥸 Self Taught #MUE | 27 \n𖥸 #beautyinfluencer...,6331,5712,59383,675,Kristen💄MUE,https://scontent.xx.fbcdn.net/v/t51.2885-15/39...,2018-09-04 22:19:55,10.281946
1,17841404101386592,sabrinamakeup,🔸Make up 💄💋\n 🔸Atendemos em domicílio \n ...,35,268,823,37,Make up por Sabrina Matos 💋💄,https://scontent.xx.fbcdn.net/v/t51.2885-15/28...,2018-09-04 22:19:58,31.171429
2,17841401557150231,asylumstar,Pro MUA. Performer. Cosplayer. Content Creator...,5164,3868,133686,2639,N O T U R W A I F U 🎀,https://scontent.xx.fbcdn.net/v/t51.2885-15/35...,2018-09-04 22:20:02,26.637103
3,17841400793636038,bonjour_beauty2020,See What's Within \nNC 👒 GA\nCosmetologist| Ma...,353,613,3909,131,Diamond,https://scontent.xx.fbcdn.net/v/t51.2885-15/27...,2018-09-04 22:20:05,12.810198
4,17841402159001280,hanskincarecosmetics,🌿 HAN: Healthy And Natural. \n💄 Helping you lo...,6130,7092,90044,873,HAN SKIN CARE COSMETICS,https://scontent.xx.fbcdn.net/v/t51.2885-15/17...,2018-09-04 22:20:27,15.846003
5,17841400270633216,michellegungoren,Sweden \n⚡When a woman's stories are \nremembe...,18337,60664,457365,157,MICHELLE 🌙,https://scontent.xx.fbcdn.net/v/t51.2885-15/36...,2018-09-04 22:20:30,28.250477
6,17841400560913493,maxmonterroso,"⚓I've been a puppet, a pauper, a pirate, a poe...",6801,39919,483423,574,Max Monterroso,https://scontent.xx.fbcdn.net/v/t51.2885-15/39...,2018-09-04 22:20:35,76.950743
7,17841401775826181,awilou,↠ 21. she/her. ITA. \n↠ snapchat: itsidax\n↠ o...,15758,16726,816151,988,Ida🍂,https://scontent.xx.fbcdn.net/v/t51.2885-15/39...,2018-09-04 22:20:39,52.854233
8,17841406827898653,kelly_younique,☆ Compte professionnel ☆\n• Maquillages et soi...,46,809,1636,38,Kelly Distributrice Younique ☄,https://scontent.xx.fbcdn.net/v/t51.2885-15/28...,2018-09-04 22:20:41,53.152174
9,17841401010512180,labricosmetics,Wear nothing but...\nLa’Bri cosmetics💄\nFounde...,138,3441,3494,79,La'Bri Cosmetics™️,https://scontent.xx.fbcdn.net/v/t51.2885-15/28...,2018-09-04 22:20:46,50.253623


In [58]:
business_discovery_all = fb_graph.get_business_discovery_all(media_count = constants.MAX_MEDIA, username = 'sabrinamakeup')

In [60]:
dict_result = fb_graph.parser_influencer_data(business_discovery_all)

In [61]:
dict_result

{'id': '17841404101386592',
 'biography': '🔸Make up 💄💋\n 🔸Atendemos  em domicílio     \n               informações: 991345989 wpp 📞',
 'followers_count': 268,
 'media_count': 37,
 'name': 'Make up por Sabrina Matos 💋💄',
 'username': 'sabrinamakeup',
 'profile_picture_url': 'https://scontent.xx.fbcdn.net/v/t51.2885-15/28152978_221577878388720_6592099119399960576_n.jpg?_nc_cat=0&oh=2c621cbdef18387f5c8e5f746e28bba8&oe=5C2663F6',
 'comments_count': 35,
 'likes_count': 823}

In [62]:
dict_result['id']

'17841404101386592'

In [55]:
df[df['username'] == 'sabrinamakeup']

,id,username,biography,comments_count,followers_count,likes_count,media_count,name,profile_picture_url,updated_date
1,17841404101386592,sabrinamakeup,updated,0,0,0,0,updated,,2018-09-03 15:02:20.402


In [63]:
# UserBusinessDiscovery.objects(id = 17841404101386592, username = 'sabrinamakeup').update(biography='updated', comments_count=0, followers_count=0, likes_count=0, media_count = 0, name = 'updated', profile_picture_url = "", updated_date = datetime.now())

UserBusinessDiscovery.objects(id = dict_result['id'], username = dict_result['username']).update(biography=dict_result['biography'], comments_count=dict_result['comments_count'], followers_count=dict_result['followers_count'], likes_count=dict_result['likes_count'], media_count = dict_result['media_count'], name = dict_result['name'], profile_picture_url = dict_result['profile_picture_url'], updated_date = datetime.now())

In [24]:
UserBusinessDiscovery.objects(id = 17841404101386592,username = 'sabrinamakeup').update(account_type = 0, categories = ['Fashion','Hairstyles','Makeup','Skincare','Beauty Blogger'])


In [192]:
rows = db.session.execute("select id, username from test.test;")
df = rows._current_rows

In [193]:
df.head()

,id,username


In [195]:
df.shape[0]

0

In [139]:
df.to_csv('data_users_business_discovery.csv', header=True, sep='\t', index=False)

In [140]:
df1 = pd.read_csv('data_users_business_discovery.csv', header=0, sep='\t')

In [38]:
rein = recommend_influencer(influencer_id=112233, ts = uuid_from_time(time.time()), influencer_name = 'Nguyen Van A', categories = ['Travel', 'Foods', 'Fashion'])
rein.save()

recommend_influencer(influencer_name='Nguyen Van A', bio=None, categories=['Travel', 'Foods', 'Fashion'], commentPerPost=None, demographics=None, engagementRate=None, follower=None, account_type=None, likePerPost=None, location=None, path=None, reach=None, website=None, posts_starting=None, currency=None, socialAccounts=[], created=datetime.datetime(2018, 9, 3, 7, 13, 1, 576642), updated=datetime.datetime(2018, 9, 3, 7, 13, 1, 576655), activate=datetime.datetime(2018, 9, 3, 7, 13, 1, 576663), influencer_id=112233, img_url=None, ts=UUID('28286fbc-af49-11e8-bd49-10948a00e8b7'))

In [55]:
info_1 = fb_graph.get_business_info(username = 'emmaparkerandco')
info_2 = fb_graph.get_business_info(username = 'natje_9999')

In [56]:
info_1['new_post'] = 0
info_2['new_post'] = 0

In [57]:
a1 = type_business_info(**info_1)
a2 = type_business_info(**info_2)

In [58]:
wl = watch_list(user_id = 'anhlbt',watch_user_list= {'natje_9999': a2, 'emmaparkerandco': a1}, viewed_time = datetime.utcnow(),updated_time = datetime.utcnow())
wl.save()

watch_list(user_id='anhlbt', watch_user_list={'natje_9999': <__main__.type_business_info object at 0x7fd7f74f8198>, 'emmaparkerandco': <__main__.type_business_info object at 0x7fd7f74f8d68>}, viewed_time=datetime.datetime(2018, 9, 4, 2, 5, 36, 941062), updated_time=datetime.datetime(2018, 9, 4, 2, 5, 36, 941064))

In [59]:
query_string="select * from %s.watch_list where user_id = 'anhlbt'"%constants.KEYSPACE_NAME
# query_string = 'truncate aify_recommend.watch_list_'
re = _session.execute(query_string)
# re.current_rows[0]

In [60]:
re.current_rows

[{'user_id': 'anhlbt',
  'updated_time': datetime.datetime(2018, 9, 4, 2, 5, 36, 941000),
  'viewed_time': datetime.datetime(2018, 9, 4, 2, 5, 36, 941000),
  'watch_user_list': OrderedMapSerializedKey([('emmaparkerandco', <__main__.type_business_info object at 0x7fd7f74f8390>), ('natje_9999', <__main__.type_business_info object at 0x7fd81c4106d8>)])}]

In [173]:
test = type_business_info(id = 'id_test',
    website = 'link web',
    media_count = 12,
    followers_count = 100,
    name = 'Tuan Anh',
    username = 'anhlbt',
    profile_picture_url = 'url_picture')

# watch_list

In [61]:
def get_wl_by_user_id(user_id):
    result = {}
    try:
        query_string="select * from watch_list where user_id = '%s'"%user_id
        re = _session.execute(query_string)
        list_dict_ = map_to_dict_factory_1(re.current_rows)#data get from db, list have only one item
    #     list_dict_ = re.current_rows
        result['status'] = 'True'
        result['data'] = convert_info_objects_to_json(list_dict_[0])
        return result
    except Exception as ex:
        result['status'] = 'False'
        return result

    
def convert_info_objects_to_json(info_object):
    tmp_dict = info_object['watch_user_list']
    for k, v in tmp_dict.items():
        print(k)
        tmp_dict[k] = dict(tmp_dict[k])
    return info_object    
    
def wl_add(user_id, ig_id, fb_graph):
    result = {}
    try:
        wl_item = fb_graph.get_business_info(username = ig_id)
        wl_item['new_post'] = 0
        wl = get_wl_by_user_id(user_id)
        print(wl)
        if wl['status'] == 'True':
            wl = wl['data']
            if wl is None:
                wl = watch_list(user_id = user_id, watch_user_list = {ig_id:type_business_info(**wl_item)}, viewed_time = datetime.utcnow(),updated_time = datetime.utcnow())
                wl.save()
                result['status'] = True
                result['data'] = wl_item # try catch here
            else:
                if wl['watch_user_list'] is None:
                    watch_list.objects(user_id = user_id).update(watch_user_list= {ig_id:type_business_info(**wl_item)})
                    result['status'] = True
                    result['data'] = wl_item
                else:   
                    if ig_id in wl['watch_user_list']:
                        print("user %s already in watch list"%ig_id)
                        result['status'] = False
                    else:
                        wl['watch_user_list'][ig_id] = type_business_info(**wl_item)     
                        watch_list.objects(user_id = user_id).update(watch_user_list= wl['watch_user_list'])
                        result['status'] = True
                        result['data'] = wl_item # try catch here
        
    except Exception as ex:
        print(ex)
        result['status'] = False
    return result    

def wl_remove(user_id, ig_id, fb_graph):
    try:
        wl = get_wl_by_user_id(user_id)
        if wl is None:
            return False
        elif wl['watch_user_list'] is not None and ig_id in wl['watch_user_list']:
            #print('remove %s '%ig_id)
            del wl['watch_user_list'][ig_id]
            watch_list.objects(user_id = user_id).update(watch_user_list= None)
            watch_list.objects(user_id = user_id).update(watch_user_list= wl['watch_user_list'])
            return True
        else:
            print('no see %s in database'%ig_id)
            return False
    except Exception as ex:
        print(ex)
        return False    
    
    

def wl_update_new_post(user_id, fb_graph):
#     list_infos = []
    try:
        wl = get_wl_by_user_id(user_id)
        if wl is None:
            return False
        elif wl['watch_user_list'] is not None and (datetime.now() - wl['updated_time']).total_seconds() > constants.TOTAL_SECONDS_TO_REFRESH_WATCH_LIST:
            for k, username in enumerate(wl['watch_user_list']):
        #     for k, v in list_dict_[0]['watch_user_list'].items(): # iter dictionary
                print(username)
                request_data = fb_graph.get_business_info(username = username)
    #             list_infos.append(request_data)viewed_time
                #
                num_new_post = request_data['media_count'] - int(wl['watch_user_list'][username].media_count)
                request_data['new_post'] = num_new_post
                if num_new_post != 0:
                    #update db
                    wl['watch_user_list'][username] = type_business_info(**request_data) #update item in watch list user
                    print('you have %s new post'%num_new_post)
                    
        else:
            print('no have any user in watch list or user request greater than 2 time in an hour ...')
            return False
                
        watch_list.objects(user_id = user_id).update(watch_user_list= wl['watch_user_list'])      

    except Exception as ex:
        print(ex)
        
# watch_list.objects(user_id = 'anhlbt').update(watch_user_list= {'natje_9999': a2, 'emmaparkerandco': a2})   

# update_string = "update watch_list set watch_user_list= {'natje_9999': 'a2', 'emmaparkerandco': 'a1'} where user_id = 'anhlbt' if exists"
# _session.execute(update_string)

# wl = watch_list(user_id = 'anhlbt3',watch_user_list= a['watch_user_list'], viewed_time = datetime.utcnow(),updated_time = datetime.utcnow())
# wl.save()

In [158]:
query_string="select * from test"
re = _session.execute(query_string)
list_dict_ = map_to_dict_factory_1(re.current_rows)#data get from db, list have only one item

In [159]:
list_dict_

[]

In [163]:
# user_id = 'anhlbt'
# query_string="select * from watch_list where user_id = '%s'"%user_id
# re = _session.execute(query_string)
# list_dict_ = map_to_dict_factory_1(re.current_rows)#data get from db, list have only one item


# query_string = "COPY test(id , biography , comments_count , followers_count , likes_count , media_count , name , profile_picture_url , updated_date , username ) FROM users_bussiness_dicsovery.csv"
# COPY users_business_discovery (id , biography , comments_count , followers_count , likes_count , media_count , name , profile_picture_url , updated_date , username ) FROM 'users_bussiness_dicsovery.csv'



In [ ]:
COPY users_business_discovery (id , biography , comments_count , followers_count , likes_count , media_count , name , profile_picture_url , updated_date , username ) FROM 'users_bussiness_dicsovery.csv'


In [187]:
a = pd.read_csv('users_business_discovery_201809051046.csv', header=0, sep = '\t', index_col = None)

In [191]:
for col in a.columns:
    print(col, type(a.iloc[1][col]))

id <class 'numpy.int64'>
username <class 'str'>
biography <class 'str'>
comments_count <class 'numpy.int64'>
followers_count <class 'numpy.int64'>
likes_count <class 'numpy.int64'>
media_count <class 'numpy.int64'>
name <class 'str'>
profile_picture_url <class 'str'>
updated_date <class 'str'>


In [165]:
wl_item = fb_graph.get_business_info(username = 'emmaparkerandco')

In [166]:
wl_item

{'id': '17841401690883742',
 'website': 'https://www.emmaparkerdiamonds.com/',
 'followers_count': 649,
 'media_count': 331,
 'name': 'Emma Parker & Co.',
 'username': 'emmaparkerandco',
 'profile_picture_url': 'https://scontent.xx.fbcdn.net/v/t51.2885-15/12393747_142605859440768_1709991462_a.jpg?_nc_cat=0&_nc_ad=z-m&_nc_cid=0&oh=f2d635c5b36f4a70850d961564c3503d&oe=5BF30F51'}

In [63]:
wl = get_wl_by_user_id('anhlbt')

emmaparkerandco
natje_9999


In [18]:
wl_add('anhlbt', 'natje_9999', fb_graph)

emmaparkerandco
natje_9999
{'status': 'True', 'data': {'user_id': 'anhlbt', 'updated_time': datetime.datetime(2018, 8, 24, 7, 6, 46, 555000), 'viewed_time': datetime.datetime(2018, 8, 24, 7, 6, 46, 555000), 'watch_user_list': {'emmaparkerandco': {'id': '17841401690883742', 'website': 'https://www.emmaparkerdiamonds.com/', 'media_count': 330, 'followers_count': 647, 'name': 'Emma Parker & Co.', 'username': 'emmaparkerandco', 'profile_picture_url': 'https://scontent.xx.fbcdn.net/v/t51.2885-15/12393747_142605859440768_1709991462_a.jpg?_nc_cat=0&oh=8b7b47ecf00a2e0e865d2c0d06993e63&oe=5BF30F51', 'new_post': 0}, 'natje_9999': {'id': '17841401660142798', 'website': 'https://bit.ly/2v0yJDU', 'media_count': 805, 'followers_count': 1696, 'name': 'Nathalie Madsen', 'username': 'natje_9999', 'profile_picture_url': 'https://scontent.xx.fbcdn.net/v/t51.2885-15/33933220_493197567763039_6170514771826180096_n.jpg?_nc_cat=0&oh=4d09397c07c3f0d0667f8688d02a4e65&oe=5BF17C73', 'new_post': 0}}}}
user natje_9

{'status': False}

In [112]:
wl_remove('anhlbt', 'natje_9999', fb_graph)

emmaparkerandco
natje_9999
'watch_user_list'


False

In [75]:
a = get_wl_by_user_id('anhlbt')

emmaparkerandco
natje_9999


In [76]:
a

{'status': 'True',
 'data': {'user_id': 'anhlbt',
  'updated_time': datetime.datetime(2018, 9, 4, 2, 5, 36, 941000),
  'viewed_time': datetime.datetime(2018, 9, 4, 2, 5, 36, 941000),
  'watch_user_list': {'emmaparkerandco': {'id': '17841401690883742',
    'website': 'https://www.emmaparkerdiamonds.com/',
    'media_count': 331,
    'followers_count': 650,
    'name': 'Emma Parker & Co.',
    'username': 'emmaparkerandco',
    'profile_picture_url': 'https://scontent.xx.fbcdn.net/v/t51.2885-15/12393747_142605859440768_1709991462_a.jpg?_nc_cat=0&oh=8b7b47ecf00a2e0e865d2c0d06993e63&oe=5BF30F51',
    'new_post': -476},
   'natje_9999': {'id': '17841401660142798',
    'website': 'https://bit.ly/2v0yJDU',
    'media_count': 807,
    'followers_count': 1675,
    'name': 'Nathalie Madsen',
    'username': 'natje_9999',
    'profile_picture_url': 'https://scontent.xx.fbcdn.net/v/t51.2885-15/33933220_493197567763039_6170514771826180096_n.jpg?_nc_cat=0&oh=4d09397c07c3f0d0667f8688d02a4e65&oe=5BF17

In [77]:
a2.name

'Nathalie Madsen'

In [78]:
a1.name

'Emma Parker & Co.'

In [70]:
watch_list.objects(user_id = 'anhlbt').update(watch_user_list= {'natje_9999': a2, 'emmaparkerandco': a2})   


In [19]:
wl_update_new_post('anhlbt', fb_graph)

no have any user in watch list or user request greater than 2 time in an hour, please add first...


False

In [81]:
(datetime.now()-a['updated_time']).total_seconds() > 3600


True

In [73]:
a1

datetime.datetime(2018, 9, 3, 13, 38, 42, 57535)

In [67]:
a2

datetime.datetime(2018, 8, 24, 7, 6, 46, 555000)

In [75]:
a3 = a1 - a2

In [76]:
print(a3)

10 days, 6:31:55.502535


In [78]:
a3.total_seconds()


887515.502535

In [182]:
a['watch_user_list']['natje_9999'].new_post

0

In [395]:
# prepared = _session.prepare("""
#         INSERT INTO aify_recommend.watch_list_ (user_id, watch_user_list, updated_time, viewed_time) 
#         VALUES(?, ?, ?, ?)
#         """)

# _session.execute(prepared.bind(("emmaparkerandco", {'annieverywhere':watch_list[0]}, datetime.utcnow(), datetime.utcnow() )))

# _session.execute(prepared.bind(("emmaparkerandco", datetime.utcnow(), datetime.utcnow(),\
#                                 {"natje_9999":{"media_count":"30", "followers_count": "65"}, "annieverywhere":{"media_count":"50", "followers_count": "65"}})))

#update map
update_string = "update watch_list_ set updated_time= %s, viewed_time = %s where user_id = 'emmaparkerandco' if exists"%(datetime.utcnow(), datetime.utcnow())
_session.execute(update_string)

In [61]:
#calculate num new post:
#first, select from db, we have: [{'anhlbt': 19}, {'longld': 100}]
#second, call api to get current data (up to date)
#then, calculate new post and update db,
fb_graph = facebook_graph_api.FacebookGraphApi('../FacebookCrawler/instagram_token.csv')
s_time = time.time()
test = fb_graph.get_business_info(username = 'emmaparkerandco')#17841401610104138, 17841407517779759
print(test)
print(time.time() - s_time)

/home/anh_lbt/atvn/sw_follou/recommender_api/test
{'id': '17841401690883742', 'website': 'https://www.emmaparkerdiamonds.com/', 'followers_count': 650, 'media_count': 330, 'name': 'Emma Parker & Co.', 'username': 'emmaparkerandco', 'profile_picture_url': 'https://scontent.xx.fbcdn.net/v/t51.2885-15/12393747_142605859440768_1709991462_a.jpg?_nc_cat=0&oh=8b7b47ecf00a2e0e865d2c0d06993e63&oe=5BF30F51'}
0.596560001373291


In [53]:
test['media_count']

330

In [242]:
_session

In [259]:
# ['id', 'biography', 'followers_count', 'media_count', 'name', 'username', 'profile_picture_url', 'comments_count', 'likes_count']


first_name = 'Anh updated'
last_name = 'Lesdfa'
table = 'Person'
# _session.execute("update Person set last_name=%s, first_name =%s where id=b9595ecc-a51e-11e8-aff2-c1049a4e205f if exists", (last_name,first_name))
# insert_string = "insert into Person (id, first_name, last_name) values (uuid_from_time(time.time()), 'Anh', 'le')"

update_string = "update Person set last_name= '%s', first_name = '%s' where id=81833564-a5bd-11e8-a36c-67d94cf8123f and user_name ='anhlbt' if exists"%(first_name, last_name)
print(update_string)
_session.execute(update_string)



update Person set last_name= 'Anh updated', first_name = 'Lesdfa' where id=81833564-a5bd-11e8-a36c-67d94cf8123f and user_name ='anhlbt' if exists


In [14]:
insert_string

"INSERT INTO aify_recommend.watch_list (user_id, newest_num_post, viewed_time, watch_user_list) VALUES('emmaparkerandco', 0, '2018-08-22 10:47:11.309934', ?);"

In [133]:
from datetime import datetime
datetime.utcnow()

datetime.datetime(2018, 8, 22, 3, 34, 6, 812638)

In [107]:
id1 = uuid_from_time(time.time())
id2 = uuid_from_time(time.time())

In [108]:
from cassandra.util import datetime_from_uuid1, unix_time_from_uuid1

In [110]:
print(unix_time_from_uuid1(id1))
print(unix_time_from_uuid1(id2))

1534845616.599598
1534845616.599598


In [104]:
toUnixTimestamp(timeuuid)

AttributeError: module 'cassandra.util' has no attribute 'toUnixTimestamp'

In [112]:
rnd = random.randint(0,12)
set_business_word = ['Entertainment', 'Fashion', 'Lifestyle', 'Food And Drinks', 'Travel And Tourism','Travel','clothing','Gym Fashion','Photography','Modeling','Fitness Photography','Comercial']
set_business_word[rnd]

In [ ]:
social_user('external_id', 'username', 'post_cnt', 'avatar_url', 'post_cnt', 'followers_cnt')

user_hashtag_setting('categories')

# onl recommend 

In [407]:
rows = db.session.execute("select * from %s.social_user;"%(constants.KEYSPACE_NAME))
rows2 = db.session.execute("select * from %s.user_hashtag_setting;"%(constants.KEYSPACE_NAME))
df1 = rows._current_rows
df2 = rows2._current_rows

In [408]:
df = df1.merge(df2[['owner_id','categories']],left_on='id', right_on='owner_id')


In [412]:
df.loc[df['id'] == '17841400363002924', 'categories'] = 'Fashion, Travel, clothes'
df.loc[df['id'] == '17841400488056752', 'categories'] = 'Fashion, Travel, Beauty'
df.loc[df['id'] == '17841403916959140', 'categories'] = 'accessories, Beauty'
df.loc[df['external_id'] == '17841400363002924', 'home_link'] = 'Fashion, Travel, clothes'

In [413]:
df.head(10)

,social_type,external_id,access_token,account_id,avatar_url,created,followers_cnt,follows_cnt,fullname,id,post_cnt,updated,username,owner_id,categories,home_link
0,instagram_business,17841400363002924,EAACLXTuiZBKkBADl1XmuR4gsNuSIGwQtTLaKcXUU9o34x...,nctam@atvn.com,https://scontent.xx.fbcdn.net/v/t51.2885-15/22...,2018-08-28 02:57:09,831760,10,Book Smarter,ba5ee910-aaa8-11e8-a25d-bbf697a6eaed,436,2018-08-28 02:57:09,booksmartertravel,ba5ee910-aaa8-11e8-a25d-bbf697a6eaed,None,"Fashion, Travel, clothes"
1,instagram_business,17841400488056752,EAACLXTuiZBKkBAIDkSQUvWSlm7ILTKMxTkpOn10T8ZB6D...,nctam@atvn.com,https://scontent.xx.fbcdn.net/v/t51.2885-15/12...,2018-08-28 02:57:09,211431,44,Luxury Travel Media,ba5f1020-aaa8-11e8-a25d-bbf697a6eaed,680,2018-08-28 02:57:09,luxwtprime,ba5f1020-aaa8-11e8-a25d-bbf697a6eaed,None,NaN
2,instagram_business,17841400835466221,EAACLXTuiZBKkBACTE19k2ZASZA2jKqTukG9uVMDZBqd5G...,test@test.vn,https://scontent.xx.fbcdn.net/v/t51.2885-15/18...,2018-08-30 06:21:35,89,361,Huynh Trung Thong,f1d72270-ac1c-11e8-9f0a-f5dca0620c08,6,2018-08-30 06:21:35,trungthong95d,f1d72270-ac1c-11e8-9f0a-f5dca0620c08,"[""hot great"",""Vehicle service and accessories""...",NaN
3,instagram_business,17841401610104138,EAACLXTuiZBKkBALZCbVrZBpkptC5tJHaDZCWxZAZBFSI2...,nctam@atvn.com,https://scontent.xx.fbcdn.net/v/t51.2885-15/35...,2018-08-28 02:57:09,2784507,450,Luxury Travel Community,ba54d6f0-aaa8-11e8-a25d-bbf697a6eaed,4426,2018-08-28 02:57:09,luxuryworldtraveler,ba54d6f0-aaa8-11e8-a25d-bbf697a6eaed,"[""Travel""]",NaN
4,instagram_business,17841403916959140,EAACLXTuiZBKkBAKOYWdHtqJ6J3Q3eTZC3FEZCDUVUxPHZ...,john@gmail.com,https://scontent.xx.fbcdn.net/v/t51.2885-15/14...,2018-08-30 14:18:48,50,36,John Schell,9ccf9300-ac5f-11e8-80f4-eb836fc406ca,7,2018-08-30 14:18:48,schell.je,9ccf9300-ac5f-11e8-80f4-eb836fc406ca,None,NaN
5,instagram_business,17841404755455889,EAACLXTuiZBKkBAEU0tmMoHySR8n3tUGZA7lY5zpXjePgo...,nctam@atvn.com,https://scontent.xx.fbcdn.net/v/t51.2885-15/16...,2018-08-28 02:57:09,9,6,IG Steps,ba5f1021-aaa8-11e8-a25d-bbf697a6eaed,8,2018-08-28 02:57:09,stepbystepinsta,ba5f1021-aaa8-11e8-a25d-bbf697a6eaed,None,NaN
6,instagram_business,17841406553883197,EAACLXTuiZBKkBAFdfQwPj1Wf7puhcRZB2iHQZA5kyUcUP...,arrive21@gmail.com,https://scontent.xx.fbcdn.net/v/t51.2885-15/26...,2018-08-30 04:11:08,18,46,Arrive Technologies,b8c94a10-ac0a-11e8-b3a2-63161aac83ec,16,2018-08-30 04:11:08,arrivetechnologiesvietnam,b8c94a10-ac0a-11e8-b3a2-63161aac83ec,None,NaN
7,instagram_business,17841406950952588,EAACLXTuiZBKkBAD0u5VagMY2I2fW1TtjTYLv2FrgOCGcm...,arrive22@gmail.com,https://scontent.xx.fbcdn.net/v/t51.2885-15/26...,2018-08-30 04:07:58,7,0,arrivetechnologiesvietnam2,478885a0-ac0a-11e8-b3a2-63161aac83ec,2,2018-08-30 04:07:58,arrivetechnologiesvietnam2,478885a0-ac0a-11e8-b3a2-63161aac83ec,None,NaN
8,instagram_business,17841407517779759,EAACLXTuiZBKkBAAlltLcfBBky8XGlwO7JSzDGAZCUeEDP...,arrive21@gmail.com,https://scontent.xx.fbcdn.net/v/t51.2885-15/30...,2018-08-30 04:11:08,222,815,undefined,b8cf6490-ac0a-11e8-b3a2-63161aac83ec,42,2018-08-30 04:11:08,vnbeachhotel,b8cf6490-ac0a-11e8-b3a2-63161aac83ec,None,NaN
9,instagram_business,17841407886728175,EAACLXTuiZBKkBAJtbRgxWzq6URjWzRL4xFfEZBQv5OfXS...,john@gmail.com,https://scontent.xx.fbcdn.net/v/t51.2885-15/32...,2018-08-30 14:18:48,2,2,FolloU Application,9c88c600-ac5f-11e8-80f4-eb836fc406ca,1,2018-08-30 14:18:48,follouapp,9c88c600-ac5f-11e8-80f4-eb836fc406ca,None,NaN


In [332]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stemmer = PorterStemmer()

def train_matrix(df, other_format = True): #build matrix to calculate distance between vector
        
    list_business = list()
    for x in range(len(df['categories'])):
        try:
            if len(df['categories'].iloc[x])> 0:
                if other_format == True:
                    list_business.append(' '.join([stemmer.stem(word) for word in df['categories'].iloc[x]]))
                else:
                    #column categories with type text (ex: '["hot great","Vehicle service and accessories","Toys and hobbies","Sports and outdoors","Vehicle sales"]')
                    list_business.append(' '.join([stemmer.stem(word) for word in re.sub(r'[\[\]\"]', "", df['categories'].iloc[x]).split(',')]))     
            else:
                print(len(df['categories'][x]))
        except Exception as ex:
            print(ex)
            pass
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(list_business)

    return tfidf_matrix, tfidf_vectorizer, list_business

In [333]:
tfidf_matrix, tfidf_vectorizer, list_business = train_matrix(df, False)

object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()
object of type 'NoneType' has no len()


In [334]:
tfidf_vectorizer.vocabulary_

{'fashion': 3,
 'travel': 12,
 'cloth': 2,
 'beauti': 1,
 'hot': 6,
 'great': 4,
 'vehicle': 13,
 'service': 9,
 'accessori': 0,
 'toys': 11,
 'hobbi': 5,
 'sports': 10,
 'outdoor': 7,
 'sal': 8}

In [367]:
#predict 
list_business_word = 'travel'

list_text = []
list_text= [stemmer.stem(x) for x in list_business_word.split(',')]
list_text = ' '.join(list_text)


sentence_to_vector = tfidf_vectorizer.transform([list_text])#toarray()
arr = cosine_similarity(sentence_to_vector, tfidf_matrix)
preds = arr[0, 0:]

sorted_inds = [i[0] for i in sorted(enumerate(-preds), key=lambda x:x[1])]

In [380]:
sorted_inds

[3, 1, 0, 2, 4]

In [381]:
[item for item in sorted_inds if preds[item]> 0.5]

[3, 1]

In [371]:
import numpy as np
a = sorted(i for i in preds if i >= 0.5)
for i in enumerate(-np.array(a)):
    print(i)

(0, -0.5062044059286201)
(1, -1.0)


In [373]:
preds

array([0.4622077 , 0.50620441, 0.        , 1.        , 0.        ])

[0.5062044059286201, 1.0]

In [351]:
sorted_inds

[3, 1, 0, 2, 4]

In [338]:
preds > 0.5

array([False,  True, False,  True, False])

In [312]:
tmp_df = df.iloc[sorted_inds]

In [329]:
tmp_df = tmp_df.loc[:, tmp_df.columns !='social_type']

In [386]:
tmp_df[:1]

,id,access_token,account_id,profile_picture_url,created,followers_count,follows_cnt,name,id_,media_count,updated_date,username,owner_id,categories,biography,comments_count,likes_count,account_type
3,17841401610104138,EAACLXTuiZBKkBALZCbVrZBpkptC5tJHaDZCWxZAZBFSI2...,nctam@atvn.com,https://scontent.xx.fbcdn.net/v/t51.2885-15/35...,2018-08-28 02:57:09,2784507,450,Luxury Travel Community,ba54d6f0-aaa8-11e8-a25d-bbf697a6eaed,4426,2018-08-28 02:57:09,luxuryworldtraveler,ba54d6f0-aaa8-11e8-a25d-bbf697a6eaed,"[""Travel""]",,,,


In [387]:
df = tmp_df

In [392]:
df = df.reindex(sorted(df.columns), axis=1)

In [396]:
df['access_token'][df['account_id'] == 'nctam@atvn.com'] = 'aaa'

3    EAACLXTuiZBKkBALZCbVrZBpkptC5tJHaDZCWxZAZBFSI2...
1    EAACLXTuiZBKkBAIDkSQUvWSlm7ILTKMxTkpOn10T8ZB6D...
0    EAACLXTuiZBKkBADl1XmuR4gsNuSIGwQtTLaKcXUU9o34x...
Name: access_token, dtype: object

In [276]:
tfidf_matrix.toarray()

array([[0.2773501, 0.2773501, 0.2773501, 0.2773501, 0.2773501, 0.2773501,
        0.2773501, 0.2773501, 0.2773501, 0.5547002]])

In [331]:
a = [3]
isinstance(a, list)

True

In [277]:
tfidf_vectorizer.vocabulary_

{'hot': 3,
 'great': 1,
 'vehicle': 9,
 'service': 6,
 'accessories': 0,
 'toys': 8,
 'hobbies': 2,
 'sports': 7,
 'outdoors': 4,
 'sales': 5}

In [250]:
"""
id bigint,
username text,
biography text,
comments_count int,
followers_count int,
likes_count int,
media_count int,
name text,
profile_picture_url text,
updated_date timestamp,
"""   

{
    "id": social_user.external_id, 
    "username": social_user.username, 
    "biography": null, 
    "comments_count": null, 
    "followers_count": social_user.followers_cnt, 
    "likes_count": null, 
    "media_count": social_user.post_cnt, 
    "name": social_user.fullname, 
    "profile_picture_url": social_user.avatar_url, 
    "updated_date": 1535138885827, social_user.updated
    "categories": user_hashtag_setting.categories, 
    "instagram_id": social_user.username, 
    "account_type": null
}

['id', 'username', 'biography', 'comments_count', 'followers_count',
       'likes_count', 'media_count', 'name', 'profile_picture_url',
       'updated_date', 'categories', 'instagram_id', 'account_type',
       'type_resource']

SyntaxError: invalid syntax (<ipython-input-250-ef2cc8eff103>, line 25)

In [268]:
column_name = ['social_type', 'id', 'access_token', 'account_id',
       'profile_picture_url', 'created', 'followers_count', 'follows_cnt', 'name',
       'id_', 'media_count', 'updated_date', 'username', 'account_type', 'owner_id', 'categories']

In [266]:
df.columns = column_name

In [267]:
df.head(1)

,social_type,id,access_token,account_id,profile_picture_url,created,followers_count,follows_cnt,name,id_,media_count,updated_date,username,owner_id,categories
0,instagram_business,17841400363002924,EAACLXTuiZBKkBADl1XmuR4gsNuSIGwQtTLaKcXUU9o34x...,nctam@atvn.com,https://scontent.xx.fbcdn.net/v/t51.2885-15/22...,2018-08-28 02:57:09,831760,10,Book Smarter,ba5ee910-aaa8-11e8-a25d-bbf697a6eaed,436,2018-08-28 02:57:09,booksmartertravel,ba5ee910-aaa8-11e8-a25d-bbf697a6eaed,None


In [269]:
df['biography'] = ''
df['comments_count'] = ''
df['likes_count'] = ''
df['account_type'] = ''

In [320]:
df = df[['id',
       'profile_picture_url', 'created', 'followers_count', 'follows_cnt', 'name',
        'media_count', 'updated_date', 'username', 'account_type', 'categories']]

In [315]:
column_name = ['social_type', 'id', 'access_token', 'account_id',
            'profile_picture_url', 'created', 'followers_count', 'follows_cnt', 'name',
            'id_', 'media_count', 'updated_date', 'username', 'owner_id', 'categories']

In [316]:
column_name

['social_type',
 'id',
 'access_token',
 'account_id',
 'profile_picture_url',
 'created',
 'followers_count',
 'follows_cnt',
 'name',
 'id_',
 'media_count',
 'updated_date',
 'username',
 'owner_id',
 'categories']

In [317]:
len(column_name)

15